<a href="https://colab.research.google.com/github/duongdqq/TF_guide/blob/master/3_train_n_evaluation_w_builtin_methods.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

# API overview: a first end-to-end example

In [ ]:
inputs = keras.Input(shape=(784,), name='digits')
x = layers.Dense(64, activation='relu', name='dense_1')(inputs)
x = layers.Dense(64, activation='relu', name='dense_2')(x)
outputs = layers.Dense(10, activation='softmax', name='predictions')(x)

model = keras.Model(inputs=inputs, outputs=outputs)

In [ ]:
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()

11493376/11490434 [==============================] - 0s 0us/step


In [ ]:
print(len(x_train))
print(len(x_test))
x_train = x_train.reshape(60000, 784).astype('float32') / 255
x_test = x_test.reshape(10000, 784).astype('float32') / 255

y_train = y_train.astype('float32')
y_test = y_test.astype('float32')

x_val = x_train[-10000:]
y_val = y_train[-10000:]
x_train = x_train[:-10000]
y_train = y_train[:-10000]

60000
10000


In [ ]:
print(len(x_train))
print(len(x_val))
print(len(x_test))
print('-'*10)
print(len(y_train))
print(len(y_val))
print(len(y_test))

50000
10000
10000
----------
50000
10000
10000


In [ ]:
# specify the training configuration
model.compile(
    optimizer=keras.optimizers.RMSprop(),
    loss=keras.losses.SparseCategoricalCrossentropy(),
    metrics=[keras.metrics.SparseCategoricalAccuracy()]
)

In [ ]:
# call fit() to slide data into batches
history = model.fit(
    x_train,
    y_train,
    batch_size=64,
    epochs=2,
    validation_data=(x_val, y_val)
) 

Epoch 1/2
782/782 [==============================] - 2s 3ms/step - loss: 0.3450 - sparse_categorical_accuracy: 0.9024 - val_loss: 0.1816 - val_sparse_categorical_accuracy: 0.9468
Epoch 2/2
782/782 [==============================] - 2s 3ms/step - loss: 0.1600 - sparse_categorical_accuracy: 0.9520 - val_loss: 0.1382 - val_sparse_categorical_accuracy: 0.9624


In [ ]:
# history holds record of the loss values and metric values during training
history.history

{'loss': [0.34497153759002686, 0.1599685698747635],
 'sparse_categorical_accuracy': [0.9023799896240234, 0.951960027217865],
 'val_loss': [0.18158414959907532, 0.1382094919681549],
 'val_sparse_categorical_accuracy': [0.9467999935150146, 0.9624000191688538]}

In [ ]:
# evaluate model on the test data
results = model.evaluate(x_val, y_val, batch_size=128)

79/79 [==============================] - 0s 2ms/step - loss: 0.1382 - sparse_categorical_accuracy: 0.9624


In [ ]:
predictions = model.predict(x_test[:3])
predictions

array([[3.48152971e-05, 1.67615945e-05, 2.05418887e-03, 2.37399642e-03,
        1.53936867e-07, 1.45703098e-05, 9.24110122e-09, 9.95478868e-01,
        3.13362739e-06, 2.33921000e-05],
       [8.17281034e-05, 2.22759227e-05, 9.99624848e-01, 2.42608672e-04,
        1.14756095e-11, 5.64695824e-07, 9.74724912e-07, 8.82700413e-10,
        2.68861440e-05, 3.28591286e-11],
       [1.01378704e-04, 9.90791202e-01, 5.84370829e-03, 7.75401073e-04,
        2.09038364e-04, 7.84161966e-04, 1.56837763e-04, 8.46703129e-04,
        4.69970430e-04, 2.15890741e-05]], dtype=float32)

In [ ]:
predictions.shape

(3, 10)

# The compile() method: specifying a loss, metrics, and an optimizer

* Specify optimizer, loss function and metrics by model.compile()

In [ ]:
inputs = keras.Input(shape=(784,), name='digits')
x = layers.Dense(64, activation='relu', name='dense_1')(inputs)
x = layers.Dense(64, activation='relu', name='dense_2')(x)
outputs = layers.Dense(10, activation='softmax', name='predictions')(x)

model = keras.Model(inputs=inputs, outputs=outputs)

* 1st method

In [ ]:
model.compile(
    optimizer=keras.optimizers.RMSprop(learning_rate=1e-3),
    loss=keras.losses.SparseCategoricalCrossentropy(),
    metrics=[keras.metrics.SparseCategoricalAccuracy()]
)

* If model has multiple outputs, specify different losses and metrics for each output, and modulate the contribution of each output to the total loss of the model

* 2nd method

In [ ]:
model.compile(
    optimizer='rmsprop',
    loss='sparse_categorical_crossentropy',
    metrics=['sparse_categorical_accuracy']
)

* For later reuse, put the model definition and compile step in function

In [ ]:
def get_uncompiled_model():
  inputs = keras.Input(shape=(784,), name='digits')
  x = layers.Dense(64, activation='relu', name='dense_1')(inputs)
  x = layers.Dense(64, activation='relu', name='dense_2')(x)
  outputs = layers.Dense(10, activation='softmax', name='predictions')(x)
  model = keras.Model(inputs=inputs, outputs=outputs)
  return model

In [ ]:
def get_compiled_model():
  model = get_uncompiled_model()
  model.compile(
    optimizer='rmsprop',
    loss='sparse_categorical_crossentropy',
    metrics=['sparse_categorical_accuracy']
  )
  return model

### Many built-in optimizers, losses, and metrics are available

* Optimizers
  * SDG() with or without momentum
  * RMSprop()
  * Adam()
* Losses
  * MeanSquaredError()
  * KLDivergence()
  * CosineSimilarity()
* Metrics
  * AUC()
  * Precision()
  * Recall()

### Custom losses

* 1st method

In [ ]:
def custom_mean_squared_error(y_true, y_pred):
  return tf.math.reduce_mean(tf.square(y_true - y_pred))

model = get_uncompiled_model()
model.compile(
    optimizer=keras.optimizers.Adam(),
    loss=custom_mean_squared_error
)

In [ ]:
y_train_one_hot = tf.one_hot(y_train, depth=10)
model.fit(x_train, y_train_one_hot, batch_size=64, epochs=1)

782/782 [==============================] - 1s 2ms/step - loss: 0.0159


* 2nd method

### Custom metrics

### Handling losses and metrics that don't fit the standard signature

### Automatically setting apart a validation holdout set

# Training & evaluation from tf.data Datasets